In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pickle
import ipywidgets as widgets
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer,make_column_selector

In [2]:
rf_pickle = open (r"C:\Training\Kaggle\Datasets\Heart Failure\rf.pickle", "rb")
best_model_rf = pickle.load(rf_pickle)
print(best_model_rf)

RandomForestClassifier(max_features=1, random_state=2022)


In [3]:
#ct_pickle = open (r"C:\Training\Kaggle\Datasets\Heart Failure\ct.pickle", "rb")
#ct = pickle.load(ct_pickle)
#print(ct.get_feature_names_out())

In [4]:
df = pd.read_csv(r"C:\Training\Kaggle\Datasets\Heart Failure\heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [5]:
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']


### Using One Hot Encoder

ohc = OneHotEncoder(handle_unknown='ignore')
ct = make_column_transformer((ohc,
       make_column_selector(dtype_include=object)),
                             ("passthrough",
                              make_column_selector(dtype_include=['int64','float64'])))
dum_np = ct.fit_transform(X)

In [6]:
def input_test(Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex,ChestPainType,RestingECG,ExerciseAngina,ST_Slope):
    tst_df = pd.DataFrame([[Age, Sex, ChestPainType, RestingBP, Cholesterol, FastingBS,
       RestingECG, MaxHR, ExerciseAngina, Oldpeak, ST_Slope]],
                          columns=['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope'])
    return tst_df;

def predict_on_test(Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex,ChestPainType,RestingECG,ExerciseAngina,ST_Slope):
    d_frame = input_test(Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex,ChestPainType,RestingECG,ExerciseAngina,ST_Slope)
    tst = pd.DataFrame(ct.transform(d_frame),columns=ct.get_feature_names_out())
    p = best_model_rf.predict_proba(tst)
    prob = widgets.BoundedFloatText(value=p[:,1][0]*100,min=0.0,
                                    max=100.0,disabled=True,
                                    description='% Risk:')
    display(prob);

In [7]:
w = widgets.interactive(predict_on_test,  
Age = widgets.IntSlider(value=53,min=28,max=77,step=1,description='Age:',disabled=False,
                         continuous_update=True,readout=True),
RestingBP = widgets.IntSlider(value=132,min=0,max=200,step=1,description='Resting BP:',disabled=False,
                         continuous_update=True,readout=True),
Cholesterol = widgets.IntSlider(value=199,min=0,max=603,step=1,description='Cholesterol:',disabled=False,
                         continuous_update=True,readout=True),
FastingBS = widgets.Checkbox(value=True,description='Fasting Sugar>120 mg/dl',disabled=False,
                         continuous_update=True,readout=True),
MaxHR = widgets.IntSlider(value=137,min=60,max=202,step=1,description='Max HR:',disabled=False,
                         continuous_update=True,readout=True),
Oldpeak = widgets.FloatSlider(value=1.0,min=-3.0,max=6.2,step=0.05,description='Old Peak:',disabled=False,
                         continuous_update=True,readout=True,readout_format='.1f'),
Sex = widgets.Dropdown(options=['Male','Female'],value='Female',description='Sex:',disabled=False,
                         continuous_update=True,readout=True),
ChestPainType = widgets.Dropdown(options=['ATA', 'NAP', 'ASY', 'TA'],value='NAP',
                                    description='Chest Pain:',disabled=False,
                         continuous_update=True,readout=True),
RestingECG = widgets.Dropdown(options=['Normal', 'ST', 'LVH'],value='Normal',
                                    description='Resting ECG:',disabled=False,
                         continuous_update=True,readout=True),
ExerciseAngina = widgets.Dropdown(options=['N', 'Y'],value='N',
                                    description='Ex. Angina:',disabled=False,
                         continuous_update=True,readout=True),
ST_Slope = widgets.Dropdown(options=['Up', 'Flat', 'Down'],value='Up',
                                    description='ST Slope:',disabled=False,
                         continuous_update=True,readout=True))



display(w)

interactive(children=(IntSlider(value=53, description='Age:', max=77, min=28), IntSlider(value=132, descriptio…